# Runtime Errors

The downside of dynamic languages like Python is runtime errors can happen more easily. Thankfully we can use AI to help us find issues especially when errors are a bit difficult.

Here is an example that can be difficult to navigate if you are not aware of some nuances in pandas/NumPy/and SciPy.

## Some Basic Examples

Can you spot what the runtime error will be? Use your AI assistant if you cannot.

In [ ]:
import pandas as pd

# Create sample data as a dictionary
data = [
    { "customer_id": 1, "name": "Alice", "total_spent" : 150.0},
    { "customer_id": 2, "name": "Bob", "total_spent" : None},
    [ "customer_id", 3, "name", "Charlie", "total_spent", 300.5]
]

# Create DataFrame
customers_df = pd.DataFrame(data)
customers_df


## Troubleshooting Linear and Logistic Regression Models

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import requests

# Download data from GitHub
url = "https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/single_independent_variable_linear_small.csv"
open("data.csv", "wb") \
    .write(requests.get(url).content)

# Load data into a DataFrame
df = pd.read_csv("data.csv")

# Extract input variables (all rows, all columns but last column)
X = df[:,:-1]

# Extract output column (all rows, last column)
Y = df[:, -1:1]

# Fit a line to the points
fit = LinearRegression().fit(X, Y)

# m = 1.7867224, b = -16.51923513
m = fit.coef_.flatten()
b = fit.intercept_.flatten()
print("m = {0}".format(m))
print("b = {0}".format(b))

# show in chart
plt.plot(X, Y, 'o') # scatterplot
plt.plot(X, m*X+b) # line
plt.show()

Let's say I just want to inline my X and Y data. Oh, now what?

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

X = [1,2,3,4,5,6,7,8,9,10]
Y =  [5,10,10,15,14,15,29,18,25,23]

# Fit a line to the points
fit = LinearRegression().fit(X, Y)

# m = 1.7867224, b = -16.51923513
m = fit.coef_.flatten()
b = fit.intercept_.flatten()
print("m = {0}".format(m))
print("b = {0}".format(b))

# show in chart
plt.plot(X, Y, 'o') # scatterplot
plt.plot(X, m*X+b) # line
plt.show()

Now let's say we are trying to make a machine learning model using a logistic regression. Something is wrong here. And even after you have the AI correct this, you may have a further issue.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import requests

# Download data from GitHub
url = "https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/classification/simple_logistic_regression.csv"
open("data.csv", "wb") \
    .write(requests.get(url).content)

# Load data into a Pandas DataFrame
df = pd.read_csv("data.csv")

# Extract input variables (all rows, all columns but last column)
X = df.values[:, :-1]

# Extract output column (all rows, last column)
Y = df.values[:, -1]

# Separate training and testing data to evaluate performance and reduce overfitting
# This leaves a third of the data out for testing
# Set a random seed just to make the randomly selected split consistent
X_train, Y_train, X_test, Y_test = train_test_split(X, Y, test_size=1.0/3.0, random_state=10)

model = LogisticRegression(penalty='none')
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)
print("R^2: %.3f" % result)


# show in chart
plt.plot(X, Y, 'o') # scatterplot
plt.plot(X, model.coef_.flatten()*X+model.intercept_.flatten())
plt.show()

## Troubleshooting a Neural Network

Below is some code to train a neural network completely from scratch in NumPy. But there are some common errors in the backpropation with the tensor shapes, which can get tricky.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Download data from GitHub
url = "https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/classification/light_dark_font_training_set.csv"
open("data.csv", "wb") \
    .write(requests.get(url).content)

data = pd.read_csv("data.csv")

# Learning rate controls how slowly we approach a solution
# Make it too small, it will take too long to run.
# Make it too big, it will likely overshoot and miss the solution.
L = 0.05

# Extract the input columns, scale down by 255
all_inputs = (data.iloc[:, 0:3].values / 255.0)
all_outputs = data.iloc[:, -1].values

# Split train and test data sets
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs,
    test_size=1 / 3)
n = X_train.shape[0]


# Build a neural network with weights and biases
# with random initialization
w_hidden = np.random.rand(3, 3)
w_output = np.random.rand(1, 3)

b_hidden = np.random.rand(3, 1)
b_output = np.random.rand(1, 1)

# Activation functions
relu = lambda x: np.maximum(x, 0)
logistic = lambda x: 1 / (1 + np.exp(-x))

# Runs inputs through the neural network to get predicted outputs
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden
    A1 = relu(Z1)
    Z2 = w_output @ A1 + b_output
    A2 = logistic(Z2)
    return Z1, A1, Z2, A2

# Derivatives of Activation functions
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# returns slopes for weights and biases
# using chain rule
def backward_prop(Z1, A1, Z2, A2, X, Y):
    dC_dA2 = 2 * A2 - 2 * Y
    dA2_dZ2 = d_logistic(Z2)
    dZ2_dA1 = w_output
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1

    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2

    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2

    dC_dA1 = dC_dA2 @ dA2_dZ2 @ dZ2_dA1

    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1

    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2

# Execute gradient descent
for i in range(100_000):
    # randomly select one of the training data
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # run randomly selected training data through neural network
    Z1, A1, Z2, A2 = forward_prop(X_sample)

    # distribute error through backpropagation
    # and return slopes for weights and biases
    dW1, dB1, dW2, dB2 = backward_prop(Z1, A1, Z2, A2, X_sample, Y_sample)

    # update weights and biases
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_output -= L * dW2
    b_output -= L * dB2

# Calculate accuracy
test_predictions = forward_prop(X_test.transpose())[3]  # grab only A2
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test)
accuracy = sum(test_comparisons.astype(int) / X_test.shape[0])
print("ACCURACY: ", accuracy)

Let's now implement a shell prompt for testing. But again, something is wrong here.

In [ ]:
# Interact and test with new colors
def predict_probability(r, g, b):
    X = np.array([r, g, b])
    Z1, A1, Z2, A2 = forward_prop(X)
    return A2

def predict_font_shade(r, g, b):
    output_values = predict_probability(r, g, b)
    if output_values > .5:
        return "DARK"
    else:
        return "LIGHT"

while True:
    col_input = input("Predict light or dark font. Input values R,G,B: ")
    (r, g, b) = col_input.split(",")
    print(predict_font_shade(int(r), int(g), int(b)))

## Brainstorm

In [ ]:
def get_middle_element(items):
    # Attempt to get the middle element of a list
    mid_index = len(items) // 2
    return items[mid_index]

# Test the function
data = []
result = get_middle_element(data)
print(result)

In [ ]:
def get_user_age(user_data):
    # Retrieve age from a user dictionary
    return user_data["age"]

# Test the function
user = {"name": "Alice", "email": "alice@example.com"}
age = get_user_age(user)
print(f"User age: {age}")

In [39]:
def build_greeting(name, score):
    # Create a greeting message with a score
    return "Hello, " + name + "! Your score is: " + score

# Test the function
user_name = "Bob"
user_score = 95
greeting = build_greeting(user_name, user_score)
print(greeting)

TypeError: can only concatenate str (not "int") to str

In [ ]:
class Product:
    def __init__(self, name, price):
        self.name = name
        self.price = price

    def get_price(self):
        return self.price

# Test the code
item = Product("Laptop", 999.99)
price = item.getPrice()  # Note the subtle typo
print(f"Price: {price}")

In [ ]:
import matplotlib.pyplot as plt

categories = ["A", "B", "C"]
values = [10, 25]

plt.bar(categories, values)
plt.title("Category Counts")
plt.xlabel("Categories")
plt.ylabel("Values")
plt.show()

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {
    "Name": ["Alice", "Bob", "Charlie"],
    "Age": [25, 30, 35]
}
df = pd.DataFrame(data)

# Attempt to access a non-existent column
result = df["name"].str.lower()  # Error: wrong column name
print(result)

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {
    "City": ["New York", "London", "Tokyo"],
    "Population": [8.4, 9.0, 37.4]
}
df = pd.DataFrame(data)

row = df[3]
print(row)

In [ ]:
import pandas as pd

# Create sample data as a dictionary
data = [
    { "customer_id": 1, "name": "Alice", "total_spent" : 150.0},
    { "customer_id": 2, "name": "Bob", "total_spent" : None},
    [ "customer_id", 3, "name", "Charlie", "total_spent", 300.5]
]

# Create DataFrame
customers_df = pd.DataFrame(data)
customers_df


In [ ]:
import pandas as pd

# Create sample data as a dictionary
data = {
    'customer_id': [1, 2, 3, 4],
    'name': ['Alice', 'Bob', 'Charlie'],
    'total_spent': [150.0, None, 300.5]
}

# Create DataFrame
customers_df = pd.DataFrame(data)

# Set customer_id as the index (similar to PRIMARY KEY)
customers_df.set_index('customer_id', inplace=True)